In [ ]:
%load_ext autoreload
%autoreload 2

# Entity resolution

In [ ]:
import pandas as pd
from pathlib import Path
from tools import eval_methods, resolution_methods
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
import random
import itertools
from tqdm import tqdm

In [ ]:
gazetteer_df = pd.read_csv("../processed/wikidata/british_isles_gazetteer.csv", header=0, index_col=0, low_memory=False)

In [ ]:
import pickle
with open("/resources/wikipedia/extractedResources/overall_entity_freq.pickle", 'rb') as fp:
    wikipedia_entity_overall_dict = pickle.load(fp)

### Feature selection

In [ ]:
# ------------------
# Feature selection for dev and test
num_candidates_list = [5, 3, 1]
settings = ["test", "dev"]
candrank_approaches = ["deezy_match"]
for num_candidates in num_candidates_list:
    for setting in settings:
        for candrank in candrank_approaches:
            features_file = "../processed/resolution/features_" + candrank + "_" + setting + str(num_candidates) + ".tsv"
            if not Path(features_file).is_file():
                df = pd.read_pickle("../processed/resolution/candranking_" + candrank + "_" + setting + str(num_candidates) + ".pkl")
                exp_df = resolution_methods.feature_selection(candrank, df, gazetteer_df, wikipedia_entity_overall_dict)
                exp_df.drop_duplicates(subset=['Query','Candidate'], inplace=True)
                exp_df.to_csv(features_file, sep="\t")
            print(candrank + " " + setting + " " + str(num_candidates) + " done!")

features_dev = pd.read_csv("../processed/resolution/features_" + candrank + "_dev" + str(num_candidates) + ".tsv",sep='\t', index_col=0)
features_test = pd.read_csv("../processed/resolution/features_" + candrank + "_test" + str(num_candidates) + ".tsv",sep='\t', index_col=0)

### Settings

In [ ]:
num_candidates = 1
candrank_method = "deezy_match"

features_dev = pd.read_csv("../processed/resolution/features_" + candrank + "_dev" + str(num_candidates) + ".tsv",sep='\t', index_col=0)

### Method and baselines

In [ ]:
# We will store the results of all methods/baselines as columns in the original structured dataframe:
results_test_df = pd.read_pickle("../processed/quicks/quicks_test.pkl")
features_test = pd.read_csv("../processed/resolution/features_" + candrank + "_test" + str(num_candidates) + ".tsv",sep='\t', index_col=0)

In [ ]:
# -------------------------------
# Apply our classification method (column "our_method_all")
# -------------------------------

# ------------------------------
# Train the classifier with all the features
dev_df = features_dev # development set feature vectors
use_cols_all = ['f_0','f_1','f_2','f_3','f_4','f_5','f_6','f_7','f_8'] # features to use
clf_all = resolution_methods.train_classifier(dev_df, use_cols_all)

# ------------------------------
# Apply the classifier with all the features
features_test_df = features_test # test set feature vectors
results_test_df = resolution_methods.our_method_simple(features_test_df, clf_all, use_cols_all, gazetteer_df, results_test_df)

In [ ]:
# -------------------------------
# Baseline: Apply candrank-most-confident baseline
# -------------------------------
results_test_df = resolution_methods.candrank_most_confident(features_test_df, results_test_df)

In [ ]:
# -------------------------------
# Baseline: Apply wikipedia-most-relevant baseline
# -------------------------------
results_test_df = resolution_methods.wikipedia_most_relevant(features_test_df, results_test_df)

In [ ]:
# -------------------------------
# Baseline: Apply semantically_most_similar baseline
# -------------------------------
results_test_df = resolution_methods.semantically_most_similar(features_test_df, results_test_df)

In [ ]:
# -------------------------------
# RankLib: Apply learning to rank (note: performed and averaged 5 runs because results fluctuated quite a lot)
# -------------------------------

code_folder = str(Path("../../").resolve()) + "/"
filter="all"
feature_combination = "allfeatures" # Uncomment if you want to use all features 
cross_val = False

# Apply all features combination to test set:
results_test_df = resolution_methods.ranklib(features_dev,features_test_df,filter,code_folder,cross_val,results_test_df,feature_combination,num_candidates)

In [ ]:
results_test_df.head()

In [ ]:
# -------------------------------
# Skyline: Best possible result given candidates
# -------------------------------
results_test_df = resolution_methods.skyline(features_test_df, results_test_df)

In [ ]:
# -------------------------------
# Our method comb: Combine stations and places classifiers
# -------------------------------

use_cols_all = ["f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8"] 

# Train railway stations classifier (exact setting)
# --------------------------------------------
dev_df = features_dev # development set feature vectors
df_exact = dev_df[dev_df["Exact"] == 1]
use_cols_stations = use_cols_all
# Train the classifier:
clf_stations = resolution_methods.train_classifier(df_exact, use_cols_all)

# Train places classifier (not exact setting)
# --------------------------------------------
dev_df = features_dev # development set feature vectors
df_inexact = dev_df[dev_df["Exact"] == 0]
use_cols_places = use_cols_all
# Train the classifier:
clf_places = resolution_methods.train_classifier(df_inexact, use_cols_all)

# Find optimal threshold for stations/placess
# --------------------------------------------
optimal_threshold = 0.0
keep_acc = 0.0
for th in np.arange(0, 1, 0.05):
    th = round(th, 2)
    results_dev_df = pd.read_pickle("../processed/quicks/quicks_dev.pkl")
    results_dev_df = resolution_methods.our_method_comb(features_dev, clf_stations, use_cols_stations, clf_places, use_cols_places, gazetteer_df, th, results_dev_df)
    acc = eval_methods.topres_exactmetrics(results_dev_df, "our_method_comb", False)
    if acc >= keep_acc:
        optimal_threshold = th
        keep_acc = acc
        
print(optimal_threshold, keep_acc)

# Apply our classification methods (column "our_method")
# -------------------------------
features_test_df = features_test # test set feature vectors
results_test_df = resolution_methods.our_method_comb(features_test_df, clf_stations, use_cols_stations, clf_places, use_cols_places, gazetteer_df, optimal_threshold, results_test_df)

### Evaluate entity resolution

In [ ]:
eval_results = []
eval_strict = False
dApproachNames = {"candrank_most_confident":"string confidence", "wikipedia_most_relevant":"wikipedia relevance", "semantically_most_similar":"semantic coherence", "our_method_all":"SVM simple", "our_method_comb":"SVM combined", "skyline": "skyline","ranklib":"ranklib all features"}
for topres_approach in ["skyline", "candrank_most_confident", "wikipedia_most_relevant", "semantically_most_similar","ranklib", "our_method_all", "our_method_comb"]:
    print(topres_approach, candrank_method, "(strict: " + str(eval_strict) + ")", "(numCands: " + str(num_candidates) + ")")
    exact_acc = eval_methods.topres_exactmetrics(results_test_df, topres_approach, eval_strict)
    acc_at = ("-", "-", "-")
    if topres_approach != "skyline":
        acc_at = eval_methods.topres_distancemetrics(gazetteer_df, results_test_df, topres_approach, eval_strict)

    eval_mode = "strict"
    if eval_strict == False:
        eval_mode = "appr"
        
    eval_results.append([eval_mode, dApproachNames[topres_approach], exact_acc, acc_at[0], acc_at[1], acc_at[2]])

tr_eval_df = pd.DataFrame(eval_results, columns = ["Eval", "Approach", "Prec", "Acc@1km", "Acc@5km", "Acc@10km"])
tr_eval_df.round(3)
print()
print(tr_eval_df.round(2).to_latex(index=False))

### Feature analysis

In [ ]:
# ------------------------------
# Inspect best features in ranklib
code_folder = str(Path("../../").resolve()) + "/"
features_folder = str(Path("supervised_ranking/features/").resolve()) + "/"
Path(features_folder).mkdir(parents=True, exist_ok=True)
Path("supervised_ranking/feature_combs/").mkdir(parents=True, exist_ok=True)
cross_val = True
for filt in ["exact", "notexact"]:
    if not Path("supervised_ranking/feature_combs/" + filt + str(num_candidates) + ".txt").is_file():
        feature_combination = resolution_methods.find_feature_comb(features_folder, filt, cross_val, code_folder, features_dev, features_test_df, results_test_df, num_candidates)